# Random Forest LambdaMART

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
import random
import pickle
from progressbar import progressbar 

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [2]:
df_train = pd.read_csv('../data/train_data.csv')

In [3]:
df_test = pd.read_csv('../data/test_data.csv')

## Feature Engineering

Some extra feature engineering for to improve the model performance. 

In [4]:
def remove_wrong_features(df_train):
    ''' There were some features present in the cleaned dataset that were based on the bookings or the clicks
        This function removes those as they cannot be used of the test set. And both training and test set should have 
        the same features.
    '''

    drop_list = list(set(list(df_train))-set(list(df_test))- set(['booking_bool', 'click_bool']))
    features_list = [feat for feat in list(df_train) if feat not in drop_list]
    df_train = df_train[features_list]
    
    return df_train

def generate_features(df):
    
    # Composite features. These are simple features that capture the relative value of a property.
    df['price_prop_rating'] = df['price_usd'] / df['prop_starrating']
    df['price_prop_review_score'] = df['price_usd'] / df['prop_review_score']
    df['price_prop1_score'] = df['price_usd'] / df['prop_location_score1']
    df['price_prop2_score'] = df['price_usd'] / df['prop_location_score2']
    df['prop_reviewXprop_rating'] = df['prop_review_score'] * df['prop_starrating']
    df['star_diff'] = df['visitor_hist_starrating'] - df['prop_starrating']
 
    # Ranked Features. For each search ID we create a ranking from 1 till n for the existing features. 
    # Since we are working with a ranking model. These type of features work quite well.
    # First we created ranked features for the basic features 
    df['price_rank'] =  df.groupby('srch_id')['price_usd'].rank()
    df['price_rating_rank'] = df.groupby('srch_id')['prop_starrating'].rank()
    df['prop_review_score_rank'] =  df.groupby('srch_id')['prop_review_score'].rank()
    df['prop1_score_rank'] =  df.groupby('srch_id')['prop_location_score1'].rank()
    df['prop2_score_rank'] =  df.groupby('srch_id')['prop_location_score2'].rank()
    
    # Then we created ranked features for the features we created ourselves
    df['log_diff_price_rank'] =  df.groupby('srch_id')['log_diff_historical_current_price'].rank()
    df['star_diff_rank'] =  df.groupby('srch_id')['star_diff'].rank()
    df['total_price_rank'] =  df.groupby('srch_id')['total_price'].rank()
    df['total_prop_location_score_rank'] =  df.groupby('srch_id')['total_prop_location_score'].rank()
    df['price_prop_rating_rank'] =  df.groupby('srch_id')['price_prop_rating'].rank()
    df['price_prop_review_score_rank'] =  df.groupby('srch_id')['price_prop_review_score'].rank()
    df['price_prop1_score_rank'] =  df.groupby('srch_id')['price_prop1_score'].rank()
    df['price_prop2_score_rank'] =  df.groupby('srch_id')['price_prop2_score'].rank()
    
    
    return df

## Model

The idea is to create a kind of random forest with the LambdaMART model. This means that we will run a number of slightly different models, which each will have a slightly different prediction. We will then average these prediction in order to get a ensemble model that is hopefully better than any single model. 

The different versions of the model are created as follows:
    
1. Randomly remove some features (just like an actual random forest)
2. Differing random seeds for the train test split*
3. Differing random seeds for the lightgbm model itself*
    
\* Unfortunately we were not able to apply any bootstrapping to the model. Instead we opted to use different random seeds to replicate the same effect.

Source: https://en.wikipedia.org/wiki/Random_forest

In [12]:
def get_categorical(features):
    ''' The lightgbm model requires categorical faetures to be listed. In order to keep track of the categorical features
        when we are randomly removing some features we create a dictionary.
    '''
    
    feature_list = features.copy()
    feature_list.remove('booking_bool')
    feature_list.remove('click_bool')
    categorical_dict = {}
    categorical_feature = [0,1,2,5,6,9,14,15,21,23,31,32,33]
    
    for i, feat in enumerate(feature_list):
        if i in categorical_feature:
            categorical_dict[feat] = 1
        else:
            categorical_dict[feat] = 0
    
    return categorical_dict


def drop_random(df_train, df_test, random_drop=5):
    ''' This function randomly removes features from the data. 
        However, some features should not be removed or else the model crashes.
    '''
    
    feature_list = list(df_train) 
    
    categorical_dict = get_categorical(feature_list)
    do_not_remove = ['srch_id', 'prop_id', 'site_id', 'booking_bool', 'click_bool']
    
    drops = 0
    while drops < random_drop:
        drop = random.choice(feature_list)
        if drop not in do_not_remove:
            feature_list.remove(drop)
            drops += 1
       
    
    df_train = df_train[feature_list]
    
    features = feature_list.copy()
    
    # The test set does not have any bookings or click. This is not the most elegant way to deal with this. I know ;)
    features.remove('booking_bool')
    features.remove('click_bool')
    df_test = df_test[features]
    
    return df_train, df_test, categorical_dict


def map_categorical(X, categorical_dict):
    ''' This function keeps track of the categorical features'''
    feature_list = list(X)
    categorical_feature = []
    
    for i, feat in enumerate(feature_list):
        if categorical_dict[feat] == 1:
            categorical_feature.append(i)
    
    return categorical_feature

In [13]:
def split(df_train):
    ''' We cannot use a standard train_test_split. In stead we split the data on srch_id
        Here we alse create a target variable where bookings weigh more that clicks. (As described in the assigment)
    '''
    
    id_list = list(df_train['srch_id'].unique())

    random.shuffle(id_list)

    train_ids = id_list[:int(len(id_list) * 0.85)]
    val_ids = id_list[int(len(id_list) * 0.85):]

    X = df_train
    y = df_train['srch_id'].to_frame()
    y['r_real'] = df_train['click_bool'].add(df_train['booking_bool'] * 4)

    X_train = X[X['srch_id'].isin(train_ids)].drop(columns=['click_bool', 'booking_bool'])
    X_val = X[X['srch_id'].isin(val_ids)].drop(columns=['click_bool', 'booking_bool'])

    y_train = y[y['srch_id'].isin(train_ids)].drop(columns=['srch_id'])
    y_val = y[y['srch_id'].isin(val_ids)].drop(columns=['srch_id'])

    return X_train, y_train, X_val, y_val

In [14]:
def get_dataset(X_train, y_train, X_val, y_val, categorical_dict):
    ''' This function creates the Dataset object for the lightgbm using the train and validation set'''

    categorical_feature = map_categorical(X_train, categorical_dict) #[0,1,2,5,6,9,14,15,21,23,31,32,33]
    
    train_data = lgbm.Dataset(X_train,
                              label=y_train,
                              categorical_feature=categorical_feature,
                              group=list(X_train.groupby('srch_id').count()['site_id']))

    val_data = lgbm.Dataset(X_val,
                            label=y_val,
                            categorical_feature=categorical_feature,
                            group=list(X_val.groupby('srch_id').count()['site_id']))
    
    return train_data, val_data

### Hyperparameters

Here we set up the hyperparameters. For the exact definitions see:
https://lightgbm.readthedocs.io/en/latest/Parameters.html

In [15]:
def get_parameters(seed):
    ''' These are the hyperparameters. Here we set the objective to lambdarank, 
        which ensure the lightgbm will be a ranking model. We also use the nDCG score as metric for model performance.
        The nDCG is only used to determine the top 5 ranking.
    '''

    parameters = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_at': 5,
        'boosting': 'gbdt',
        'num_threads': 14, # Adjust to the number of threads you have on your PC
        'max_leaves': 40,
        'bagging_fraction': 0.85,
        'bagging_freq': 5,
        'feature_fraction': 0.98,
        'learning_rate': 0.075,
        'verbose': -1, # unfortunately I was not able to stop the model from printing every result.
        'min_data_in_leaf': 28,
        'ignore_column': 0,
        'label_gain': [0,1,2,3,4,5],
        'seed': seed
    }
    
    return parameters

In [50]:
def sort_prediction(df_test, prediction, version, i='final'):
    '''Our model outputs a values between 1 and -1. 
        These are then be sorted to create the final prediction that is uploaded to Kaggle.
    '''
    
    # Reshaping the variables to make sure we don't run into errors
    prediction = prediction.reshape((len(prediction),1))
    srch = df_test.srch_id.values.reshape((len(df_test),1))
    prop = df_test.prop_id.values.reshape((len(df_test),1))
    
    # Storing the features in a dataframe and save the intermediate result.
    results = pd.DataFrame(np.concatenate([srch, prop, prediction], axis=1), columns=['srch_id', 'prop_id', 'probabilities'])
    results.to_csv('../predictions/predictions_proba_stack_'+version+'_'+str(i)+'.csv', index=False)
    results['srch_id'] = results['srch_id'].astype(int)
    results['prop_id'] = results['prop_id'].astype(int)
    
    # Sorting the predictions first by srch_id than by the probabilities
    results = results.sort_values(by=['srch_id', 'probabilities'], ascending=False)
    results = results.drop(columns=['probabilities'])
    
    # Saving the final result and also showing a preview to make sure everything is ok.
    print(results.head(10))
    results.to_csv('../predictions/predictions_stack_'+version+'_'+str(i)+'.csv', index=False)
    
    
def get_prediction(df_test, model, version, i):
    ''' This function generates the predictions for the individual models and then calls sort_prediction'''
    prediction = model.predict(df_test)    
    sort_prediction(df_test, prediction, version, i)
    
    return prediction

### LambdaMART implemented

Here we run the actual model and we call all the previously defined functions. 
Here is an overview of what is happening:

1. We remove faulty features (remove_wrong_features)
2. We generate some extra features (generate_features)
3. We initalize a dictionary to keep track of the individual model performances. (model_dict)
4. We initialize an empty array in which we shall store the individual model predictions (predictions)
5. We implement a `try` function to make save we store our results in case of an error.
6. We loop n times. The index of the loop is then used a the random seed.
7. Random features are dropped.
8. Data is splitted into training set en validation set
9. The hyperparameters are initialized with a random seed.
10. The lightGBM starts training. 
11. The individual model results are stored in `model_dict` and the model is saved
12. The model is then used to create predictions and these predictions are stores in the `predictions` array.
13. The stored predictions are averaged by row and then sorted to get teh final prediction. This is then also saved.
14. The `model_dict` is returned

In [17]:
def model_stack(train, test, iterations):
    ''' This is the random forest LambdaMART model implemented. See description above.'''
    
    version = 'v6'
    
    train = remove_wrong_features(train)
    
    train = generate_features(train)
    test = generate_features(test)
    
    model_dict = {}
    predictions = np.zeros((len(test),iterations))
    
    try:
        for i in progressbar(range(iterations)):
            random.seed(i)
            df_train, df_test, categorical_dict = drop_random(train, test, random_drop=5)

            X_train, y_train, X_val, y_val = split(df_train)
            train_data, val_data = get_dataset(X_train, y_train, X_val, y_val, categorical_dict)
            parameters = get_parameters(seed=i)

            model = lgbm.train(parameters,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=5000,
                       early_stopping_rounds=80)

            model_dict[i] = [model.best_iteration, model.best_score['valid_0']['ndcg@5'], version, list(X_train)]
            pickle.dump(model, open( "../models/model_stack_"+version+"_"+str(i)+".pkl", "wb" ) )

            prediction = get_prediction(df_test, model, version, i)
            predictions[:,i] = prediction

        mean_predictions = predictions.mean(axis=1)
        sort_prediction(df_test, mean_predictions, version)
        pickle.dump(model_dict, open( "../models/model_dict_"+version+".pkl", "wb" ) )
        
    except Exception as e:
        pickle.dump(model_dict, open( "../models/model_dict_"+version+".pkl", "wb" ) )
        print(e)
    
    return model_dict

In [18]:
# Here we pick the number of iterations for our model.
iterations = 40
model_dict = model_stack(df_train, df_test, iterations)

C:\Users\srema\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\srema\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\srema\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A

[1]	valid_0's ndcg@5: 0.341109
Training until validation scores don't improve for 80 rounds.
[2]	valid_0's ndcg@5: 0.349442
[3]	valid_0's ndcg@5: 0.351377
[4]	valid_0's ndcg@5: 0.354136
[5]	valid_0's ndcg@5: 0.354955
[6]	valid_0's ndcg@5: 0.358812
[7]	valid_0's ndcg@5: 0.362542
[8]	valid_0's ndcg@5: 0.366805
[9]	valid_0's ndcg@5: 0.36851
[10]	valid_0's ndcg@5: 0.370939
[11]	valid_0's ndcg@5: 0.374123
[12]	valid_0's ndcg@5: 0.376309
[13]	valid_0's ndcg@5: 0.377809
[14]	valid_0's ndcg@5: 0.379858
[15]	valid_0's ndcg@5: 0.381003
[16]	valid_0's ndcg@5: 0.382062
[17]	valid_0's ndcg@5: 0.38324
[18]	valid_0's ndcg@5: 0.384581
[19]	valid_0's ndcg@5: 0.385342
[20]	valid_0's ndcg@5: 0.386094
[21]	valid_0's ndcg@5: 0.38707
[22]	valid_0's ndcg@5: 0.387576
[23]	valid_0's ndcg@5: 0.388154
[24]	valid_0's ndcg@5: 0.388711
[25]	valid_0's ndcg@5: 0.389526
[26]	valid_0's ndcg@5: 0.38972
[27]	valid_0's ndcg@5: 0.39048
[28]	valid_0's ndcg@5: 0.391087
[29]	valid_0's ndcg@5: 0.391592
[30]	valid_0's ndcg@5: 0

[252]	valid_0's ndcg@5: 0.41271
[253]	valid_0's ndcg@5: 0.412666
[254]	valid_0's ndcg@5: 0.412702
[255]	valid_0's ndcg@5: 0.412698
[256]	valid_0's ndcg@5: 0.412702
[257]	valid_0's ndcg@5: 0.412607
[258]	valid_0's ndcg@5: 0.412611
[259]	valid_0's ndcg@5: 0.41254
[260]	valid_0's ndcg@5: 0.412519
[261]	valid_0's ndcg@5: 0.412592
[262]	valid_0's ndcg@5: 0.412486
[263]	valid_0's ndcg@5: 0.412533
[264]	valid_0's ndcg@5: 0.412535
[265]	valid_0's ndcg@5: 0.412593
[266]	valid_0's ndcg@5: 0.412586
[267]	valid_0's ndcg@5: 0.412618
[268]	valid_0's ndcg@5: 0.412669
[269]	valid_0's ndcg@5: 0.4127
[270]	valid_0's ndcg@5: 0.412642
[271]	valid_0's ndcg@5: 0.41265
[272]	valid_0's ndcg@5: 0.412656
[273]	valid_0's ndcg@5: 0.412664
[274]	valid_0's ndcg@5: 0.412648
[275]	valid_0's ndcg@5: 0.412708
[276]	valid_0's ndcg@5: 0.412715
[277]	valid_0's ndcg@5: 0.412707
[278]	valid_0's ndcg@5: 0.412664
[279]	valid_0's ndcg@5: 0.412596
[280]	valid_0's ndcg@5: 0.412654
[281]	valid_0's ndcg@5: 0.412604
[282]	valid_0's

  5% (1 of 18) |#                        | Elapsed Time: 0:04:43 ETA:   1:20:27

[1]	valid_0's ndcg@5: 0.340894
Training until validation scores don't improve for 80 rounds.
[2]	valid_0's ndcg@5: 0.347251
[3]	valid_0's ndcg@5: 0.353114
[4]	valid_0's ndcg@5: 0.355952
[5]	valid_0's ndcg@5: 0.358801
[6]	valid_0's ndcg@5: 0.360874
[7]	valid_0's ndcg@5: 0.362246
[8]	valid_0's ndcg@5: 0.36554
[9]	valid_0's ndcg@5: 0.368226
[10]	valid_0's ndcg@5: 0.369378
[11]	valid_0's ndcg@5: 0.372427
[12]	valid_0's ndcg@5: 0.374258
[13]	valid_0's ndcg@5: 0.375831
[14]	valid_0's ndcg@5: 0.376906
[15]	valid_0's ndcg@5: 0.377885
[16]	valid_0's ndcg@5: 0.379099
[17]	valid_0's ndcg@5: 0.3797
[18]	valid_0's ndcg@5: 0.379907
[19]	valid_0's ndcg@5: 0.381018
[20]	valid_0's ndcg@5: 0.381479
[21]	valid_0's ndcg@5: 0.38226
[22]	valid_0's ndcg@5: 0.383432
[23]	valid_0's ndcg@5: 0.383806
[24]	valid_0's ndcg@5: 0.384401
[25]	valid_0's ndcg@5: 0.385135
[26]	valid_0's ndcg@5: 0.3856
[27]	valid_0's ndcg@5: 0.38596
[28]	valid_0's ndcg@5: 0.386628
[29]	valid_0's ndcg@5: 0.386857
[30]	valid_0's ndcg@5: 0.3

...

[252]	valid_0's ndcg@5: 0.413137
[253]	valid_0's ndcg@5: 0.413162
[254]	valid_0's ndcg@5: 0.41318
[255]	valid_0's ndcg@5: 0.413135
[256]	valid_0's ndcg@5: 0.41307
[257]	valid_0's ndcg@5: 0.413384
[258]	valid_0's ndcg@5: 0.413316
[259]	valid_0's ndcg@5: 0.413374
[260]	valid_0's ndcg@5: 0.413352
[261]	valid_0's ndcg@5: 0.413338
[262]	valid_0's ndcg@5: 0.41331
[263]	valid_0's ndcg@5: 0.413439
[264]	valid_0's ndcg@5: 0.413474
[265]	valid_0's ndcg@5: 0.413394
[266]	valid_0's ndcg@5: 0.413449
[267]	valid_0's ndcg@5: 0.413537
[268]	valid_0's ndcg@5: 0.413514
[269]	valid_0's ndcg@5: 0.413542
[270]	valid_0's ndcg@5: 0.413539
[271]	valid_0's ndcg@5: 0.413507
[272]	valid_0's ndcg@5: 0.41349
[273]	valid_0's ndcg@5: 0.413559
[274]	valid_0's ndcg@5: 0.413539
[275]	valid_0's ndcg@5: 0.413444
[276]	valid_0's ndcg@5: 0.413448
[277]	valid_0's ndcg@5: 0.413423
[278]	valid_0's ndcg@5: 0.413493
[279]	valid_0's ndcg@5: 0.413479
[280]	valid_0's ndcg@5: 0.413487
[281]	valid_0's ndcg@5: 0.413381
[282]	valid_0'

 94% (17 of 18) |######################  | Elapsed Time: 1:38:50 ETA:   0:04:51

[1]	valid_0's ndcg@5: 0.346585
Training until validation scores don't improve for 80 rounds.
[2]	valid_0's ndcg@5: 0.358455
[3]	valid_0's ndcg@5: 0.359424
[4]	valid_0's ndcg@5: 0.359999
[5]	valid_0's ndcg@5: 0.361814
[6]	valid_0's ndcg@5: 0.365421
[7]	valid_0's ndcg@5: 0.367471
[8]	valid_0's ndcg@5: 0.371899
[9]	valid_0's ndcg@5: 0.375345
[10]	valid_0's ndcg@5: 0.378701
[11]	valid_0's ndcg@5: 0.381321
[12]	valid_0's ndcg@5: 0.383341
[13]	valid_0's ndcg@5: 0.384336
[14]	valid_0's ndcg@5: 0.386185
[15]	valid_0's ndcg@5: 0.386712
[16]	valid_0's ndcg@5: 0.38824
[17]	valid_0's ndcg@5: 0.38888
[18]	valid_0's ndcg@5: 0.389303
[19]	valid_0's ndcg@5: 0.390473
[20]	valid_0's ndcg@5: 0.391109
[21]	valid_0's ndcg@5: 0.391886
[22]	valid_0's ndcg@5: 0.392243
[23]	valid_0's ndcg@5: 0.392795
[24]	valid_0's ndcg@5: 0.393974
[25]	valid_0's ndcg@5: 0.39417
[26]	valid_0's ndcg@5: 0.394647
[27]	valid_0's ndcg@5: 0.394875
[28]	valid_0's ndcg@5: 0.394526
[29]	valid_0's ndcg@5: 0.395816
[30]	valid_0's ndcg@5:

[252]	valid_0's ndcg@5: 0.417317
[253]	valid_0's ndcg@5: 0.417302
[254]	valid_0's ndcg@5: 0.417273
[255]	valid_0's ndcg@5: 0.417285
[256]	valid_0's ndcg@5: 0.417209
[257]	valid_0's ndcg@5: 0.417224
[258]	valid_0's ndcg@5: 0.417216
[259]	valid_0's ndcg@5: 0.417332
[260]	valid_0's ndcg@5: 0.417411
[261]	valid_0's ndcg@5: 0.417416
[262]	valid_0's ndcg@5: 0.417475
[263]	valid_0's ndcg@5: 0.417474
[264]	valid_0's ndcg@5: 0.417476
[265]	valid_0's ndcg@5: 0.41741
[266]	valid_0's ndcg@5: 0.417399
[267]	valid_0's ndcg@5: 0.417315
[268]	valid_0's ndcg@5: 0.417316
[269]	valid_0's ndcg@5: 0.417284
[270]	valid_0's ndcg@5: 0.417228
[271]	valid_0's ndcg@5: 0.417104
[272]	valid_0's ndcg@5: 0.417068
[273]	valid_0's ndcg@5: 0.417101
[274]	valid_0's ndcg@5: 0.417078
[275]	valid_0's ndcg@5: 0.417005
[276]	valid_0's ndcg@5: 0.416887
[277]	valid_0's ndcg@5: 0.416888
[278]	valid_0's ndcg@5: 0.416805
[279]	valid_0's ndcg@5: 0.416798
[280]	valid_0's ndcg@5: 0.416838
[281]	valid_0's ndcg@5: 0.41702
[282]	valid_

100% (18 of 18) |########################| Elapsed Time: 1:43:43 Time:  1:43:43


In [19]:
# Here we print the model_dict results to show the individual model performance

for key, value in model_dict.items():
    print(key, value[:3])

22 [249, 0.41276632912482625, 'v5']
23 [281, 0.4129883348477892, 'v5']
24 [365, 0.4074122972313243, 'v5']
25 [329, 0.4121483088886432, 'v5']
26 [199, 0.41752327037531534, 'v5']
27 [427, 0.41830720845062186, 'v5']
28 [302, 0.4135735676961655, 'v5']
29 [242, 0.4144923059042425, 'v5']
30 [306, 0.4171717349633142, 'v5']
31 [314, 0.4200280193296645, 'v5']
32 [291, 0.4147133611928614, 'v5']
33 [310, 0.4131216913780223, 'v5']
34 [318, 0.4111726092238697, 'v5']
35 [361, 0.41708072018216197, 'v5']
36 [319, 0.4203327637124458, 'v5']
37 [317, 0.41606896407324534, 'v5']
38 [273, 0.41355864038302576, 'v5']
39 [264, 0.41747615751567446, 'v5']
